<a href="https://colab.research.google.com/github/Daeun-Danna-Lee/NLP-Sentiment-Classification/blob/main/Movie_Review_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이다은/2019312492/0.828

### 개발 환경

- 애플 실리콘(M1) 맥북
- macOS Big Sur 12.0.1
- python 3.7.12
- google colab

In [ ]:
# !conda install scikit-learn

In [ ]:
# %pip install -U scikit-learn

In [30]:
!pip install konlpy

In [31]:
import sys

print(sys.version)

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [32]:
import pandas as pd

In [33]:
# 데이터 불러오기

data = pd.read_csv('./processed_data', sep=',', names=['발화문', '평균감정'])
data

,발화문,평균감정
0,"어, 청소 니가 대신 해 줘!",neutral
1,둘 다 청소 하기 싫어. 귀찮아.,neutral
2,둘 다 하기 싫어서 화내.,angry
3,그럼 방세는 어떡해.,sadness
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sadness
...,...,...
43986,나 엘리베이터에 갇혔어.,sadness
43987,하지만 기분이 나쁜 걸 어떡해?,sadness
43988,자취방 엘리베이턴데 정전인가봐.,sadness
43989,나 드디어 프로젝트 끝났어!,sadness


In [39]:
# 구둣점 제거 -> 공백으로 대체
data['발화문'] = data['발화문'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")

In [40]:
data

,발화문,평균감정
0,어 청소 니가 대신 해 줘,neutral
1,둘 다 청소 하기 싫어 귀찮아,neutral
2,둘 다 하기 싫어서 화내,angry
3,그럼 방세는 어떡해,sadness
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고,sadness
...,...,...
43986,나 엘리베이터에 갇혔어,sadness
43987,하지만 기분이 나쁜 걸 어떡해,sadness
43988,자취방 엘리베이턴데 정전인가봐,sadness
43989,나 드디어 프로젝트 끝났어,sadness


In [41]:
# train, test data 분리

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size = 0.3)

In [42]:
train_df.shape

(30793, 2)

In [43]:
print(train_df.isnull().values.any()) # 데이터에 null 값이 있는지 검사 (있으면 True 반환)

False


In [44]:
# stopwords 정의 (https://www.ranks.nl/stopwords/korean)

stopwords = ['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로', '바꾸어서 말하면', '바꾸어서 한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에 가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한 까닭에', '하기 때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느곳', '더군다나', '하물며', '더욱이는', '어느때', '언제', '야', '이봐', '어이', '여보시오', '흐흐', '흥', '휴', '헉헉', '헐떡헐떡', '영차', '여차', '어기여차', '끙끙', '아야', '앗', '아야', '콸콸', '졸졸', '좍좍', '뚝뚝', '주룩주룩', '솨', '우르르', '그래도', '또', '그리고', '바꾸어말하면', '바꾸어말하자면', '혹은', '혹시', '답다', '및', '그에 따르는', '때가 되어', '즉', '지든지', '설령', '가령', '하더라도', '할지라도', '일지라도', '지든지', '몇', '거의', '하마터면', '인젠', '이젠', '된바에야', '된이상', '만큼어찌됏든', '그위에', '게다가', '점에서 보아', '비추어 보아', '고려하면', '하게될것이다', '일것이다', '비교적', '좀', '보다더', '비하면', '시키다', '하게하다', '할만하다', '의해서', '연이서', '이어서', '잇따라', '뒤따라', '뒤이어', '결국', '의지하여', '기대여', '통하여', '자마자', '더욱더', '불구하고', '얼마든지', '마음대로', '주저하지 않고', '곧', '즉시', '바로', '당장', '하자마자', '밖에 안된다', '하면된다', '그래', '그렇지', '요컨대', '다시 말하자면', '바꿔 말하면', '즉', '구체적으로', '말하자면', '시작하여', '시초에', '이상', '허', '헉', '허걱', '바와같이', '해도좋다', '해도된다', '게다가', '더구나', '하물며', '와르르', '팍', '퍽', '펄렁', '동안', '이래', '하고있었다', '이었다', '에서', '로부터', '까지', '예하면', '했어요', '해요', '함께', '같이', '더불어', '마저', '마저도', '양자', '모두', '습니다', '가까스로', '하려고하다', '즈음하여', '다른', '다른 방면으로', '해봐요', '습니까', '했어요', '말할것도 없고', '무릎쓰고', '개의치않고', '하는것만 못하다', '하는것이 낫다', '매', '매번', '들', '모', '어느것', '어느', '로써', '갖고말하자면', '어디', '어느쪽', '어느것', '어느해', '어느 년도', '라 해도', '언젠가', '어떤것', '어느것', '저기', '저쪽', '저것', '그때', '그럼', '그러면', '요만한걸', '그래', '그때', '저것만큼', '그저', '이르기까지', '할 줄 안다', '할 힘이 있다', '너', '너희', '당신', '어찌', '설마', '차라리', '할지언정', '할지라도', '할망정', '할지언정', '구토하다', '게우다', '토하다', '메쓰겁다', '옆사람', '퉤', '쳇', '의거하여', '근거하여', '의해', '따라', '힘입어', '그', '다음', '버금', '두번째로', '기타', '첫번째로', '나머지는', '그중에서', '견지에서', '형식으로 쓰여', '입장에서', '위해서', '단지', '의해되다', '하도록시키다', '뿐만아니라', '반대로', '전후', '전자', '앞의것', '잠시', '잠깐', '하면서', '그렇지만', '다음에', '그러한즉', '그런즉', '남들', '아무거나', '어찌하든지', '같다', '비슷하다', '예컨대', '이럴정도로', '어떻게', '만약', '만일', '위에서 서술한바와같이', '인 듯하다', '하지 않는다면', '만약에', '무엇', '무슨', '어느', '어떤', '아래윗', '조차', '한데', '그럼에도 불구하고', '여전히', '심지어', '까지도', '조차도', '하지 않도록', '않기 위하여', '때', '시각', '무렵', '시간', '동안', '어때', '어떠한', '하여금', '네', '예', '우선', '누구', '누가 알겠는가', '아무도', '줄은모른다', '줄은 몰랏다', '하는 김에', '겸사겸사', '하는바', '그런 까닭에', '한 이유는', '그러니', '그러니까', '때문에', '그', '너희', '그들', '너희들', '타인', '것', '것들', '너', '위하여', '공동으로', '동시에', '하기 위하여', '어찌하여', '무엇때문에', '붕붕', '윙윙', '나', '우리', '엉엉', '휘익', '윙윙', '오호', '아하', '어쨋든', '만 못하다하기보다는', '차라리', '하는 편이 낫다', '흐흐', '놀라다', '상대적으로 말하자면', '마치', '아니라면', '쉿', '그렇지 않으면', '그렇지 않다면', '안 그러면', '아니었다면', '하든지', '아니면', '이라면', '좋아', '알았어', '하는것도', '그만이다', '어쩔수 없다', '하나', '일', '일반적으로', '일단', '한켠으로는', '오자마자', '이렇게되면', '이와같다면', '전부', '한마디', '한항목', '근거로', '하기에', '아울러', '하지 않도록', '않기 위해서', '이르기까지', '이 되다', '로 인하여', '까닭으로', '이유만으로', '이로 인하여', '그래서', '이 때문에', '그러므로', '그런 까닭에', '알 수 있다', '결론을 낼 수 있다', '으로 인하여', '있다', '어떤것', '관계가 있다', '관련이 있다', '연관되다', '어떤것들', '에 대해', '이리하여', '그리하여', '여부', '하기보다는', '하느니', '하면 할수록', '운운', '이러이러하다', '하구나', '하도다', '다시말하면', '다음으로', '에 있다', '에 달려 있다', '우리', '우리들', '오히려', '하기는한데', '어떻게', '어떻해', '어찌됏어', '어때', '어째서', '본대로', '자', '이', '이쪽', '여기', '이것', '이번', '이렇게말하자면', '이런', '이러한', '이와 같은', '요만큼', '요만한 것', '얼마 안 되는 것', '이만큼', '이 정도의', '이렇게 많은 것', '이와 같다', '이때', '이렇구나', '것과 같이', '끼익', '삐걱', '따위', '와 같은 사람들', '부류의 사람들', '왜냐하면', '중의하나', '오직', '오로지', '에 한하다', '하기만 하면', '도착하다', '까지 미치다', '도달하다', '정도에 이르다', '할 지경이다', '결과에 이르다', '관해서는', '여러분', '하고 있다', '한 후', '혼자', '자기', '자기집', '자신', '우에 종합한것과같이', '총적으로 보면', '총적으로 말하면', '총적으로', '대로 하다', '으로서', '참', '그만이다', '할 따름이다', '쿵', '탕탕', '쾅쾅', '둥둥', '봐', '봐라', '아이야', '아니', '와아', '응', '아이', '참나', '년', '월', '일', '령', '영', '일', '이', '삼', '사', '오', '육', '륙', '칠', '팔', '구', '이천육', '이천칠', '이천팔', '이천구', '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '령', '영']


In [45]:
# 토큰화

from nltk.tokenize import word_tokenize
from konlpy.tag import Kkma, Okt
from konlpy.tag import Okt
from tqdm import tqdm

train_text = []
test_text = []

okt = Okt()
for text in tqdm(train_df['발화문']):
  temp = okt.morphs(text, stem=True) # 형태소 기준 토큰화, 원형으로 변형
  no_stopwords_ver = [word for word in temp if not word in stopwords] # 스탑워드 제거
  train_text.append(no_stopwords_ver)
for text in tqdm(test_df['발화문']):
  temp = okt.morphs(text, stem=True)
  no_stopwords_ver = [word for word in temp if not word in stopwords]
  test_text.append(no_stopwords_ver)
train_label = train_df['평균감정']
test_label = test_df['평균감정']

100%|██████████| 13198/13198 [00:22<00:00, 599.58it/s]


In [ ]:
train_text

In [46]:
import sklearn

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def identity_tokenizer(text):
    return text

vetorizer = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, ngram_range=(1, 1))    
x_train = vetorizer.fit_transform(train_text)
x_test = vetorizer.transform(test_text)

In [48]:
x_train = x_train.toarray()
feature_names = vetorizer.get_feature_names()
print(feature_names)
print(x_train)

['ㄹ', 'ㅊ', 'ㅏ', '가게', '가격', '가계부', '가기', '가까이', '가깝다', '가끔', '가끔가다', '가끔가다가', '가나', '가능성', '가능하다', '가다', '가다가', '가도', '가득', '가득하다', '가든', '가뜩이나', '가라', '가라앉다', '가라앉히다', '가래', '가량', '가로수길', '가르다', '가르치다', '가만', '가만히', '가망', '가면', '가방', '가버리다', '가벼워지다', '가볍다', '가보', '가보다', '가분', '가선', '가성', '가슴', '가시', '가식', '가야', '가운데', '가이드', '가입', '가장', '가전제품', '가져가다', '가져오다', '가족', '가줄', '가지', '가지다', '가짜', '각오', '간', '간다', '간단하다', '간도', '간만', '간소', '간식', '간신히', '간의', '간전', '간정', '간직', '간촐하', '간호', '갇히다', '갈구다', '갈다', '갈라', '갈라고', '갈래', '갈리다', '갈수', '갈수록', '갈아', '감', '감격', '감고', '감기', '감다', '감사하다', '감수', '감시', '감싸다', '감염', '감염병', '감정', '갑', '갑갑하다', '갑자기', '갑작스럽다', '갑작스레', '값', '강남', '강남역', '강도', '강력하다', '강아지', '강하다', '갖다', '같앴는데', '같앴어', '갚다', '개', '개념', '개다', '개발', '개새끼', '개운하다', '개월', '개인', '개인주의', '개인플레이', '개판', '갱년기', '갱신', '걍', '걔', '걔네', '걘', '거', '거고', '거기', '거나', '거니', '거동', '거두다', '거들다', '거래', '거래처', '거려', '거르', '거리', '거린데', '거만', '거부', '거실', '거야', '거여', '거절', '거제', '거제도', '거주', '거즘', '거지', '

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [49]:
# x_train = train_text
# x_test = test_text
y_train = train_label
y_test = test_label

In [50]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))


F1 = 0.6958973983077696
accuracy = 0.7218517957266253


In [51]:
# 가장 효율적인 alpha 값 구하기
max_alpha = 0
max_accuracy = 0.0

for alpha in range(0, 300):
  alpha /= 100
  clf = MultinomialNB(alpha=alpha)
  clf.fit(x_train, y_train)
  pred = clf.predict(x_test)
  accuracy = metrics.accuracy_score(y_test, pred)
  if accuracy > max_accuracy:
    max_accuracy = accuracy
    max_alpha = alpha
  print('alpha =', alpha, 'accuracy =', accuracy)

print('max alpha =', max_alpha, 'max_accuary =', max_accuracy)


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


alpha = 0.0 accuracy = 0.7135171995756933
alpha = 0.01 accuracy = 0.7187452644340051
alpha = 0.02 accuracy = 0.7195787240490984
alpha = 0.03 accuracy = 0.7201091074405213
alpha = 0.04 accuracy = 0.7207152598878618
alpha = 0.05 accuracy = 0.7210183361115321
alpha = 0.06 accuracy = 0.7214729504470374
alpha = 0.07 accuracy = 0.7216244885588725
alpha = 0.08 accuracy = 0.7217760266707077
alpha = 0.09 accuracy = 0.7217002576147902
alpha = 0.1 accuracy = 0.7218517957266253
alpha = 0.11 accuracy = 0.7223064100621306
alpha = 0.12 accuracy = 0.7212456432792848
alpha = 0.13 accuracy = 0.721548719502955
alpha = 0.14 accuracy = 0.7218517957266253
alpha = 0.15 accuracy = 0.7216244885588725
alpha = 0.16 accuracy = 0.7218517957266253
alpha = 0.17 accuracy = 0.7219275647825428
alpha = 0.18 accuracy = 0.7220033338384604
alpha = 0.19 accuracy = 0.7216244885588725
alpha = 0.2 accuracy = 0.7211698742233672
alpha = 0.21 accuracy = 0.7210183361115321
alpha = 0.22 accuracy = 0.7205637217760267
alpha = 0.23 ac